<a href="https://colab.research.google.com/github/wyxuan02/GenerativeAI_class2025/blob/main/AI08_RAG02%E6%89%93%E9%80%A0%E5%90%91%E9%87%8F%E8%B3%87%E6%96%99%E5%BA%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

課程範例：https://github.com/yenlung/AI-Demo/blob/master/%E3%80%90Demo06b%E3%80%91RAG02_%E6%89%93%E9%80%A0_RAG_%E7%B3%BB%E7%B5%B1.ipynb

In [ ]:
# prompt: 用 Linux 指令，讀入一個在網路上，網址是 URL 的檔案。

URL = "https://drive.google.com/uc?export=download&id=1VdM8HwgDczeThp3S8MKPU8xrO784JPIv"

!wget -O faiss_db.zip "$URL"


In [ ]:
!unzip faiss_db.zip

### 1. 安裝並引入必要套件

In [ ]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from openai import OpenAI
import gradio as gr

### 2. 自訂 E5 embedding 類別

In [ ]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 3. 載入 `faiss_db`

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

### 4. 設定好我們要的 LLM

In [ ]:
import os
from google.colab import userdata

In [ ]:
api_key = userdata.get('Groq')

In [ ]:
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
model = "llama3-70b-8192"
base_url="https://api.groq.com/openai/v1"

In [ ]:
client = OpenAI(
    base_url=base_url # 使用 OpenAI 本身不需要這段
)

### 5. `prompt` 設計

In [ ]:
system_prompt = '''你是一位專業且非常有名氣的私廚，專門根據使用者的飲食偏好與健康目標提供創意餐食建議。
請確保推薦的餐點適合不同的飲食需求，如低碳水、蛋白質高、素食或低熱量選項,並且以「你的專屬私廚已上線!👨🏻‍🍳✨」結尾。
請確保你的回答僅包含餐食推薦，不提供非飲食相關的資訊。避免推薦過度加工食品，並以天然食材為主。若使用者沒有提供具體要求，請推薦一般均衡飲食的餐食。
如果使用者提供飲食偏好、過敏資訊或健康目標（如減重、增肌、控糖），請根據這些條件調整建議內容。如果使用者沒有提供具體條件，請提供一份適合大多數人的健康晚餐選擇。
請在回答時提供完整的餐點建議，包括食材、製作方式和營養價值。如果可以，提供替代選項，例如如果某種食材不適合，建議其他類似的選擇。
請用台灣習慣的繁體中文回應。所有提供的訊息請都用繁體中文回應。'''

prompt_template = """
根據下列資料回答問題：
{retrieved_chunks}

使用者的問題是：{question}

請根據資料內容回覆，若資料不足請告訴使用者敬請期待更新菜單。
"""

### 6. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [ ]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history
    # 取回相關資料
    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((user_input, answer))
    return answer

### 7. 用 Gradio 打造 Web App

In [ ]:
# 自訂 CSS

custom_css = """
body, html {
    background-color: #003377 !important;  /* 湛藍色 */
    margin: 0;
    padding: 0;
}
.gradio-container {
    background-color: #003377 !important;  /* 確保容器背景湛藍色 */
    box-shadow: none !important;  /* 移除內部陰影 */
}
h1 {
    font-size: 80px;
    font-weight: bold;
    color: 	#FFDD55;
    text-align: center;
}
p {
    font-size: 25px;
    font-weight: bold;
    color: "Black"; /* 白色文字 */
    text-align: center;
    margin-top: 20px;
}
@keyframes neon-glow {
    0% { box-shadow: 0px 0px 5px #00ffcc; }
    50% { box-shadow: 0px 0px 20px #00ffcc; }
    100% { box-shadow: 0px 0px 5px #00ffcc; }
}


input, textarea {
    background-color: #1e1e1e;  /* 深色背景 */
    color: #ffcc00;  /* 顯眼的亮黃色 */
    border: 2px solid #ffcc00; /* 黃色邊框 */
    font-size: 18px;
    padding: 10px;
    border-radius: 5px;
}
button {
    background-color: #00ffcc;
    color: #121212;
    font-weight: bold;
    border-radius: 10px;
    padding: 10px 20px;
    border: none;
    transition: all 0.3s ease-in-out;
    box-shadow: 0px 0px 15px #00ffcc; /* 霓虹藍光 */
}
button:hover {
    background-color: #ff00ff;
    color: #ffffff;
    box-shadow: 0px 0px 20px #ff00ff; /* 變成霓虹粉色 */
}
"""

In [ ]:
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("# 你的專屬私廚👨🏻‍🍳💝")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入你的問題...")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import nbformat

# 設定 notebook 檔案路徑（換成你找到的完整路徑）
notebook_path = "/content/drive/MyDrive/Colab Notebooks/AI08_RAG02打造向量資料庫.ipynb"

# 讀取 notebook
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# 移除所有輸出與 widgets metadata
for cell in nb.cells:
    if "outputs" in cell:
        cell["outputs"] = []
    if "execution_count" in cell:
        cell["execution_count"] = None

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

# 儲存回 notebook
with open(notebook_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("✅ 已移除 widgets metadata 與輸出，可以安全 push 到 GitHub！")


✅ 已移除 widgets metadata 與輸出，可以安全 push 到 GitHub！
